In [8]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from tqdm import tqdm
import re
from webdriver_manager.chrome import ChromeDriverManager

# ChromeDriver 설치 및 실행
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://eminwon.icbp.go.kr/emwp/gov/mogaha/ntis/web/emwp/cmmpotal/action/EmwpMainMgtAction.do')

def wait_for_element(selector, timeout=10):
    try:
        return WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.CSS_SELECTOR, selector)))
    except Exception as e:
        print(f"오류 발생: {e}, {selector}")
        return None

def click_element(selector, timeout=10):
    try:
        element = WebDriverWait(driver, timeout).until(EC.element_to_be_clickable((By.CSS_SELECTOR, selector)))
        element.click()
        return True
    except Exception as e:
        print(f"클릭 오류 발생: {e}, {selector}")
        return False

def element_exists(selector):
    try:
        driver.find_element(By.CSS_SELECTOR, selector)
        return True
    except:
        return False

def clean_text(html_content):
    cleaned_content = re.sub(r'<a[^>]*>.*?</a>', '', html_content, flags=re.DOTALL)
    cleaned_content = re.sub(r'<strong[^>]*>.*?</strong>', '', cleaned_content, flags=re.DOTALL)
    cleaned_text = re.sub(r'<[^>]+>', '', cleaned_content)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text

def init_data(i):
    data = {}
    data['number'] = wait_for_element(f'.table > tbody:nth-child(3) > tr:nth-child({i}) > td:nth-child(1)').text
    data['title'] = wait_for_element(f'.table > tbody:nth-child(3) > tr:nth-child({i}) > td:nth-child(2)').text
    data['name'] = wait_for_element(f'.table > tbody:nth-child(3) > tr:nth-child({i}) > td:nth-child(3)').text
    data['date'] = wait_for_element(f'.table > tbody:nth-child(3) > tr:nth-child({i}) > td:nth-child(4)').text
    data['answer_state'] = wait_for_element(f'.table > tbody:nth-child(3) > tr:nth-child({i}) > td:nth-child(5)').text
    return data

def scrape_data(data):
    try:
        content_element = wait_for_element('body > main > div > fieldset > table > tbody > tr:nth-child(4) > td')
        data['contents'] = clean_text(content_element.get_attribute('innerHTML'))
        data['has_attachment'] = element_exists('body > main > div > fieldset > table > tbody > tr:nth-child(4) > td > strong')
        
        print("data['answer_state']",data['answer_state'])
        if data['answer_state'] == '이송이첩' or data['answer_state'] =='다부처병렬':
            data['answer_contents'] = None
            data['charge'] = wait_for_element('body > main > div > table > tbody > tr:nth-child(1) > td:nth-child(2)').text
            data['answer_date'] = None
            data['registration_number'] = None
            data['charge_name'] = None
        else :        
            answer_element = wait_for_element('body > main > div > table > tbody > tr:nth-child(3) > td')
            data['answer_contents'] = clean_text(answer_element.get_attribute('innerHTML')) if answer_element else None
            data['registration_number'] = wait_for_element('body > main > div > table > tbody > tr:nth-child(1) > td:nth-child(6)').text
            data['charge_name'] = wait_for_element('body > main > div > table > tbody > tr:nth-child(2) > td:nth-child(2)').text
            data['charge'] = wait_for_element('body > main > div > table > tbody > tr:nth-child(1) > td:nth-child(2)').text
            data['answer_date'] = wait_for_element('body > main > div > table > tbody > tr:nth-child(1) > td:nth-child(4)').text

        return data
    except Exception as e:
        print(f"데이터 스크랩 중 오류 발생: {e}")
        return None

time.sleep(20)

# '민원 조회' 버튼 클릭
minwon_button = driver.find_element(By.CSS_SELECTOR, '#header > div.gnbmiddle > div > div.topmenu > ul > li:nth-child(1) > a')
minwon_button.click()

time.sleep(5)

# '공개 상담민원 조회' 버튼 클릭
gonggae_minwon_button = driver.find_element(By.CSS_SELECTOR, '#allmenu > div > div:nth-child(2) > div:nth-child(2) > ul > li:nth-child(2) > a')
gonggae_minwon_button.click()
time.sleep(10)


# 데이터 저장 리스트 초기화
data_list = []

# 페이지 순회 및 데이터 추출
for p in tqdm(range(1, 1010)):
    page_found = False
    
    while not page_found:
        page_bar = driver.find_elements(By.CSS_SELECTOR, '#navigator > .navi')
        
        for button in page_bar:
            if button.text == str(p):
                if click_element(f'#navigator > .navi:nth-child({page_bar.index(button) + 1})'):
                    page_found = True
                else:
                    print(f'{p} 페이지로 이동 실패, 재시도 중...')
                break
        
        if not page_found:
            print(f'{p} 페이지로 이동 실패, 재시도 중...')
            time.sleep(2)
    
    list_of_title = driver.find_elements(By.CSS_SELECTOR, '.td-list')
    
# id : f'.table > tbody:nth-child(3) > tr:nth-child({i}) > td:nth-child(1)'
# list :  f'.table > tbody:nth-child(3) > tr:nth-child({i}) > td:nth-child(2)'
# name : f'.table > tbody:nth-child(3) > tr:nth-child({i}) > td:nth-child(3)'
# date : f'.table > tbody:nth-child(3) > tr:nth-child({i}) > td:nth-child(4)'
# answer : f'.table > tbody:nth-child(3) > tr:nth-child({i}) > td:nth-child(5)'
    for i in range(1, len(list_of_title) + 1):
        data = init_data(i)    

        try:
            page_minwon_list = wait_for_element(f'#dataSetTb > table > tbody > tr:nth-child({i}) > td.td-list > a')
            if page_minwon_list:
                page_minwon_list.click()
                print(f'{p} 페이지, {i}번 민원 게시글 클릭 완료')
                
                scrape_data(data)    

                if data:
                    data_list.append(data)

                driver.back()
                time.sleep(1)
        except Exception as e:
            print(f"항목 {i} 처리 중 오류 발생: {e}")
            driver.back()
            time.sleep(1)

    if p % 10 == 0:
        while not click_element('#navigator > a.navi.navi-arrow.navi-arrow-single-right'):
            print(f'{p} 페이지에서 다음 페이지로 이동 실패, 재시도 중...')
            time.sleep(2)

    print(f'{p}번째 순회 중')

# 데이터 프레임으로 변환 및 저장
df = pd.DataFrame(data_list)
df.to_json('scraped_data.csv', index=False)

driver.quit()


  0%|          | 0/1009 [00:00<?, ?it/s]

1 페이지, 1번 민원 게시글 클릭 완료
data['answer_state'] 처리중
(2024-07-30 18:00:00)
1 페이지, 2번 민원 게시글 클릭 완료
data['answer_state'] 이송이첩
1 페이지, 3번 민원 게시글 클릭 완료
data['answer_state'] 처리중
(2024-07-29 18:00:00)
1 페이지, 4번 민원 게시글 클릭 완료
data['answer_state'] 처리중
(2024-07-24 18:00:00)
1 페이지, 5번 민원 게시글 클릭 완료
data['answer_state'] 처리중
(2024-07-24 18:00:00)
1 페이지, 6번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
1 페이지, 7번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
1 페이지, 8번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
1 페이지, 9번 민원 게시글 클릭 완료
data['answer_state'] 이송이첩
1 페이지, 10번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
1 페이지, 11번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
1 페이지, 12번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
1 페이지, 13번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
1 페이지, 14번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
1 페이지, 15번 민원 게시글 클릭 완료
data['answer_state'] 처리중
(2024-07-25 18:00:00)
1 페이지, 16번 민원 게시글 클릭 완료
data['answer_state'] 처리중
(2024-07-25 18:00:00)
1 페이지, 17번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
1 페이지, 18번 민원 게시글 클릭 완료
data['ans

  0%|          | 1/1009 [00:31<8:51:37, 31.64s/it]

1번째 순회 중
2 페이지, 1번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
2 페이지, 2번 민원 게시글 클릭 완료
data['answer_state'] 이송이첩
2 페이지, 3번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
2 페이지, 4번 민원 게시글 클릭 완료
data['answer_state'] 이송이첩
2 페이지, 5번 민원 게시글 클릭 완료
data['answer_state'] 이송이첩
2 페이지, 6번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
2 페이지, 7번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
2 페이지, 8번 민원 게시글 클릭 완료
data['answer_state'] 이송이첩
2 페이지, 9번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
2 페이지, 10번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
2 페이지, 11번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
2 페이지, 12번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
2 페이지, 13번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
2 페이지, 14번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
2 페이지, 15번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
2 페이지, 16번 민원 게시글 클릭 완료
data['answer_state'] 이송이첩
2 페이지, 17번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
2 페이지, 18번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
2 페이지, 19번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
2 페이지, 20번 민원 게시글 클릭 완료
data['answer_state'] 답변완료


  0%|          | 2/1009 [01:24<12:16:03, 43.86s/it]

...

5번째 순회 중
6 페이지, 1번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
6 페이지, 2번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
6 페이지, 3번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
6 페이지, 4번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
6 페이지, 5번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
6 페이지, 6번 민원 게시글 클릭 완료
data['answer_state'] 다부처병렬
오류 발생: Message: 
, body > main > div > table > tbody > tr:nth-child(3) > td
오류 발생: Message: 
, body > main > div > table > tbody > tr:nth-child(1) > td:nth-child(6)
데이터 스크랩 중 오류 발생: 'NoneType' object has no attribute 'text'
6 페이지, 7번 민원 게시글 클릭 완료
data['answer_state'] 이송이첩
6 페이지, 8번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
6 페이지, 9번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
6 페이지, 10번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
6 페이지, 11번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
6 페이지, 12번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
6 페이지, 13번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
6 페이지, 14번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
6 페이지, 15번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
6 페이지, 16번 민원 게시글 클릭 완료
d

  1%|          | 6/1009 [06:03<16:29:40, 59.20s/it]

6번째 순회 중
7 페이지, 1번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
7 페이지, 2번 민원 게시글 클릭 완료
오류 발생: Message: 
, body > main > div > fieldset > table > tbody > tr:nth-child(4) > td
데이터 스크랩 중 오류 발생: 'NoneType' object has no attribute 'get_attribute'
7 페이지, 3번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
7 페이지, 4번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
7 페이지, 5번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
7 페이지, 6번 민원 게시글 클릭 완료
data['answer_state'] 다부처병렬
오류 발생: Message: 
, body > main > div > table > tbody > tr:nth-child(3) > td
오류 발생: Message: 
, body > main > div > table > tbody > tr:nth-child(1) > td:nth-child(6)
데이터 스크랩 중 오류 발생: 'NoneType' object has no attribute 'text'
7 페이지, 7번 민원 게시글 클릭 완료
data['answer_state'] 이송이첩
7 페이지, 8번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
7 페이지, 9번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
7 페이지, 10번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
7 페이지, 11번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
7 페이지, 12번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
7 페이지, 13번 민원 게시글 클릭 완료
data['answer_state'] 답

  1%|          | 7/1009 [09:27<29:42:07, 106.71s/it]

7번째 순회 중
8 페이지, 1번 민원 게시글 클릭 완료
data['answer_state'] 이송이첩
8 페이지, 2번 민원 게시글 클릭 완료
data['answer_state'] 답변완료
8 페이지, 3번 민원 게시글 클릭 완료
data['answer_state'] 이송이첩


In [7]:
df = pd.DataFrame(data_list)
df.head(20)

,number,title,name,date,answer_state,contents,has_attachment,answer_contents,registration_number,charge_name,charge,answer_date
0,568990,밤골공원 음수대 처리재요청의 建.,이○○,2024-07-22,처리중\n(2024-07-30 18:00:00),밤골공원 음수대와 관련하여 저번 답변을 믿고기다렸으나 안내문만 기존에 있던거 다시 ...,True,귀하의 민원은 담당부서(도시관리국 공원녹지과)에서 처리중입니다.처리기한은 2024-...,202407221520250580,방은희,도시관리국 공원녹지과,
1,568452,"역무관 정은선,사회복무요원 이재호씨 칭찬합니다,",임○○,2024-07-20,이송이첩,저희 아버지께서 치매가 있으신데 부평구청역 지나시다가 급하셔서 큰볼일을 실수하셨어요...,False,None,None,None,※ 연계처리내역 : 민원상세보기(원본)\n * 추천기관 : 인천광역시 교통공사\n ...,None
2,567976,일신동 일신놀이공원 어린이물놀이장 관련,여○○,2024-07-19,처리중\n(2024-07-29 18:00:00),안녕하세요? 저는 일신동 주민입니다. 우선 일신동 일신놀이공원이 아이들을 위한 어린...,False,귀하의 민원은 담당부서(도시관리국 공원녹지과)에서 처리중입니다.처리기한은 2024-...,202407190854259564,정영범,도시관리국 공원녹지과,
3,567306,마을버스내 노인 낙상 사고,이○○,2024-07-16,처리중\n(2024-07-24 18:00:00),교통사고 피해자인 만 88세 어머니의 이야기를 듣고 아들인 제가 앞으로도 이런 사고...,False,귀하의 민원은 담당부서(안전교통국 교통행정과)에서 처리중입니다.처리기한은 2024-...,202407161356308892,이은혜,안전교통국 교통행정과,
4,567267,부평시장역 다이소 앞 도로 점거,김○○,2024-07-16,처리중\n(2024-07-24 18:00:00),"부평시장역 앞 다이소에서 매일 아침, 도로를 점거하여 사용하고 있습니다. 부평시장역...",True,귀하의 민원은 담당부서(안전교통국 주차지도과)에서 처리중입니다.처리기한은 2024-...,202407161057428853,김태형,안전교통국 주차지도과,
5,567252,바닥분수 애완동물 사용,김관옥,2024-07-16,답변완료,2024년 07월 06일 분수공원 삼산동 458-4에 아이들과 놀러 갔습니다. 11...,False,1. 구정업무에 관심을 가져주셔서 감사합니다.2. 귀하께서 제기하신 민원에 대한 답...,202407160946468838,문소정,도시관리국 공원녹지과,2024-07-16 15:14:28
6,567249,백마장-산곡동 부근 굴포천 공사장 출근시간 교통체증 해결해주십시오.,김○○,2024-07-16,답변완료,산곡동에서 백마장(부평시장 및 부평구청) 넘어가는길 아침마다 출근길 전쟁입니다. 차...,True,1. 구정 발전에 협조해 주시는 귀하께 깊이 감사드립니다.2. 귀하께서 접수하여 주...,202407160915168835,박서영,도시관리국 도시재생과,2024-07-22 14:56:28
7,567009,교통 반사경 설치,신동철,2024-07-15,답변완료,경인 대로 cj 물류 센터 출입구 반사경이 새로 설치 된 것 같은데 반사경 방향이 ...,True,1. 구정 업무에 대한 귀하의 깊은 관심에 감사드립니다.2. 귀하께서 정비 요청하신...,202407151058148595,김라희,안전교통국 교통행정과,2024-07-16 13:51:49
8,566637,부평경찰서앞 삼성전자 건물 좌회전시 불편사항,김○○,2024-07-13,이송이첩,부평경찰서앞 삼성전자 건물 좌회전시 불편사항. 신호에 따라 좌회전하는데 부평구청역 ...,False,None,None,None,※ 연계처리내역 : 민원상세보기(원본)\n * 추천기관 : 인천광역시\n * 이송(...,None
9,565612,부평 5동 중부동 공원 해충이 너무 많습니다. 지금 꽃매미 새끼~성충 너무 많아요,이○○,2024-07-10,답변완료,제목 그대로 공원쪽에서 해충이 매장안으로 너무 유입이 많이 되서 영업하기가 불편합니...,False,1. 귀하의 가정에 건강과 행복이 가득하시길 기원합니다.2. 귀하께서 제기하신 중부...,202407101254057195,김이정,도시관리국 공원녹지과,2024-07-16 11:21:36
